In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [25]:
ratings = pd.read_csv('book_rental_datasets/book_ratings.csv')
books = pd.read_csv('book_rental_datasets/books.csv')
recommend = pd.read_csv('book_rental_datasets/Recommend.csv')
users = pd.read_csv('book_rental_datasets/users.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
recommend.head()

,196,242,3,881250949
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [27]:
ratings.head()

,user_id,isbn,rating
0,276725.0,034545104X,0
1,276726.0,155061224,5
2,276727.0,446520802,0
3,276729.0,052165615X,3
4,276729.0,521795028,6


In [26]:
users.head()

,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [4]:
books.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [5]:

books = books.rename(columns = {'book-title':'title', 'book-author':'author', 'year-of-publication':'year', 'publisher':'publisher'})
users = users.rename(columns = {'User-ID':'user_id', 'Location':'location', 'Age':'age'})
ratings = ratings.rename(columns = {'User-ID':'user_id', 'Book-Rating':'rating'})

In [6]:
ratings['user_id'].value_counts()

11676.0     13602
198711.0     7550
153662.0     6109
98391.0      5891
35859.0      5850
            ...  
105002.0        1
105008.0        1
105014.0        1
105023.0        1
123969.0        1
Name: user_id, Length: 95520, dtype: int64

In [7]:
x = ratings['user_id'].value_counts() > 200
y = x[x].index #list of user_ids
print(y.shape)
ratings = ratings[ratings['user_id'].isin(y)] #give each row in ratings where the .isin(y) function returns true

(815,)


In [8]:
x[x].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [9]:
y

Float64Index([ 11676.0, 198711.0, 153662.0,  98391.0,  35859.0, 212898.0,
              278418.0,  76352.0, 110973.0, 235105.0,
              ...
               88793.0,  33145.0, 116122.0, 155916.0, 188951.0,   9856.0,
               73681.0,  59727.0,  44296.0,  28634.0],
             dtype='float64', length=815)

In [10]:
ratings_books_merged = ratings.merge(books, on='isbn')
ratings_books_merged.head() 

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher
0,277427.0,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363.0,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676.0,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538.0,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552.0,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc


In [11]:
num_rating = ratings_books_merged.groupby('book_title')['rating'].count().reset_index()
num_rating.rename(columns = {'rating':'num_ratings'}, inplace = True)
num_rating.head()

,book_title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [12]:
final_rating = ratings_books_merged.merge(num_rating, on='book_title')
final_rating.shape

(446881, 8)

In [13]:
final_rating = final_rating[final_rating['num_ratings'] >= 50]
final_rating = final_rating.drop_duplicates(['user_id', 'book_title'])
final_rating.head()

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher,num_ratings
0,277427.0,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
1,3363.0,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
2,11676.0,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
3,12538.0,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74
4,13552.0,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,74


In [14]:
books_pivot = final_rating.pivot_table(columns = 'user_id', index='book_title', values = 'rating')
books_pivot = books_pivot.fillna(0)
books_pivot

user_id,254.0,2276.0,2766.0,2977.0,3363.0,3757.0,4017.0,4385.0,6242.0,6251.0,...,249628.0,249862.0,249894.0,250184.0,250405.0,250764.0,277427.0,277478.0,277639.0,278418.0
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
books_sparse = csr_matrix(books_pivot)

In [16]:
model = NearestNeighbors(algorithm='brute')
model.fit(books_sparse)

NearestNeighbors(algorithm='brute')

In [17]:
distances, suggestions = model.kneighbors(books_pivot.iloc[11, :].values.reshape(1, -1))

In [18]:
for i in range(len(suggestions)):
    print(books_pivot.index[suggestions[i]])

Index(['A Heartbreaking Work of Staggering Genius', 'A Civil Action',
       'CAT'S EYE', 'Pleading Guilty', 'The Most Wanted'],
      dtype='object', name='book_title')


In [19]:
distances, suggestions = model.kneighbors(books_pivot.iloc[11, :].values.reshape(1, -1))

In [24]:
# Make predictions based on user and item variables
distances, suggestions = model.kneighbors(books_pivot.iloc[3, :].values.reshape(1, -1))

# for i in range(len(suggestions)):
    # print(book_pivot.index[suggestions[i]])

threshold = 5.0

list_of_titles = books_pivot.index.tolist()
list_user_id = books_pivot.columns.tolist()
    
for user in list_user_id[:50]:
    user_ratings = final_rating.loc[(final_rating["rating"] >= threshold) & (final_rating["user_id"] == user), ["isbn"]]
    
    if user_ratings.empty:
        print(f"User {user} has not liked (rated {threshold} or higher) enough books to make a suggestion.")
        continue
        
    books_rated = user_ratings["isbn"].tolist()
    suggestions = []
    
    book_titles = books["book_title"].loc[books["isbn"].isin(books_rated)].tolist()
    
    for b in book_titles:
        if b not in list_of_titles:
            continue
        else:
            index = list_of_titles.index(b)
            d, s = model.kneighbors(books_pivot.iloc[index, :].values.reshape(1, -1))
            for x in s[0]:
                if x not in suggestions and x not in books_rated:
                    suggestions.append(x)
    
    if len(suggestions) > 0:
        print("")
        print(f"Our algorithm suggests the following books for user {user}:")
        for i in suggestions:
            print(f"    {books_pivot.index[i]}")
    else:
        print("")
        print(f"User {user} has no recommendations due to an insufficient amount of ratings for the books they liked.")


Our algorithm suggests the following books for user 254.0:
    The Golden Compass (His Dark Materials, Book 1)
    CAT'S EYE
    Pleading Guilty
    The Most Wanted
    A Civil Action
    The Subtle Knife (His Dark Materials, Book 2)
    The Cardinal of the Kremlin (Jack Ryan Novels)
    Last Man Standing
    Animal Farm
    Foucault's Pendulum
    The Cradle Will Fall
    Winter Moon
    The Secret Life of Bees
    Under the Tuscan Sun
    The Saving Graces: A Novel
    She's Come Undone (Oprah's Book Club)
    Mother of Pearl
    Sleepers
    American Gods
    1984
    Harry Potter and the Sorcerer's Stone (Book 1)
    Plum Island
    Truly, Madly Manhattan
    A Man in Full
    The Bonesetter's Daughter
    The Hours : A Novel
    Moo
    The Weight of Water : A Novel Tag: Author of Resistance and Strange Fits of Passion
    Harry Potter and the Chamber of Secrets (Book 2)
    Harry Potter and the Goblet of Fire (Book 4)
    Harry Potter and the Prisoner of Azkaban (Book 3)
    Har